# Yonhap News Crawling
    
연합뉴스 기사 데이터를 장기간 아카이브하기 위한 크롤링 프로젝트입니다.    
본 저장소는 여러 가지 크롤링 시도를 통해 안정적인 수집 방안을 마련하고,    
최종적으로 Playwright + Stealth 모드 기반 크롤러를 활용하여 10년치 이상의 기사를 수집할 수 있도록 구성되었습니다.
    
---
    
### Features

* 연합뉴스 기사 크롤링 및 본문, 메타데이터(발행일, 기자, 카테고리 등) 추출
* 중복 방지 및 병렬 처리로 효율적 데이터 수집
* 발행 시각 정밀 추출을 통한 시계열 분석 지원
* .jsonl 형식 저장 지원 (추후 분석 파이프라인과 연계 가능)
    
---
    
### 필수 패키지
    
* playwright
* beautifulsoup4
* requests
* tqdm
* concurrent.futures

---
    
### Known Issues
    
* 연합뉴스는 자동화 차단 정책이 강력함
* Headless 브라우저 환경에서는 차단될 가능성 높음
* IP 차단을 피하기 위한 프록시, 분산 환경 고려 필요


# 연합뉴스 크롤링 과정 및 회고록

## 연합뉴스를 선택하게 된 이유

- 연합뉴스는 대한민국의 대표적인 뉴스 통신사로, 국내 신문·방송 등 다수 매체에 기사·사진 등을 광범위하게 공급한다.      
이는 단일 출처 기준으로도 보도 품질과 형식의 일관성을 기대할 수 있습니다.
- 연합뉴스는 북한 전담 보도 조직(N.K. news desk)을 갖추고 있으며,     
2009년 김정은 후계 체제 관련 단독 보도로 한국기자협회 대상을 수상하는 등 북한 관련 심층 취재 실적을 보유하고 있어     
북한·안보 축의 자료 수집에 적합하다고 판단했습니다.


#### 전체 개요
연합뉴스 기사를 크롤링하기 위해 여러 가지 방식을 시도하였다.    
시도 과정에서의 장점과 한계를 정리하고, 최종적으로 어떤 방법이 안정적으로 동작했는지를 기록하였다.    
    
---

### 접근 방법과 시행착오

1. RSS 활용
* 장점 : 구조가 단순하고 접근이 쉽다.
* 한계 : 과거 기사 10년치 이상은 확보가 불가하고 최신 기사만 유지되기 때문에 장기 아카이브 수집에는 부적합하다고 판단하였다.

2. Selenium과 BeautifulSoup 활용
* 시도 : 검색 페이지에서 기사 주소와 본문을 추출하였다.
* 대응 : 사용자와 유사하게 보이도록 User-Agent를 설정하고 딜레이를 삽입하였다.
* 한계 : 연합뉴스에서 자동화 접속을 로봇으로 인식하여 차단하였다.     
로딩은 가능했으나 실제 본문 접근 시 빈 페이지가 나타나거나 리디렉션이 발생하였고, 검색 페이지의 구조 또한 불안정하게 로딩되는 경우가 많았다.

3. requests와 BeautifulSoup 활용
* 시도 : 기사 주소에 직접 접근하여 본문을 파싱하였다.
* 한계 : 서버에서 차단하여 403 오류가 발생하거나 빈 결과가 반환되었고, 페이지 구조의 변화로 인해 안정적인 수집이 어려웠다.

4. 네이버 뉴스 경유
* 시도 : 네이버 뉴스 검색에서 “연합뉴스 + 북한” 조건으로 검색하여 연합뉴스 기사만 선별하였다.
* 한계 : 기사 본문 접근 시 차단되었고, 검색 결과 내에서 실제 연합뉴스 기사 주소를 추출하기가 쉽지 않았다.

5. 환경에 따른 차이
* 관찰 : 로컬 환경에서 일반 브라우저를 이용할 때는 정상적으로 접근할 수 있었다.     
그러나 Headless 브라우저 환경에서는 대부분 차단이 발생하였다.    
또한 크롬 브라우저에서는 차단되었으나 엣지 브라우저에서는 정상적으로 접근이 가능하였다.

6. Playwright와 Stealth 모드 활용
* 시도 : Playwright를 사용하고 로봇 탐지를 피할 수 있는 Stealth 모드를 적용하였다.
* 결과 : 연합뉴스 기사에 정상적으로 접근하여 본문을 파싱하는 데 성공하였다.
* 결론 : 현재까지 안정적으로 동작하는 유일한 방법임을 확인하였다.

---

### 구현 과정 (파일별 개선 흐름)

1. crawl.py (최초 버전)
* 연합뉴스 공식 사이트맵을 기반으로 크롤링하였다.
* 장점 : 과거 기사까지 수집할 수 있었고, 본문과 발행일, 기자, 카테고리 등 메타데이터를 추출할 수 있었다.
* 한계 : 단일 스레드 기반이라 속도가 느렸으며, 병렬 처리나 중복 방지 기능이 미흡하였다.
    
     
2. crawl1.py (1차 개선)
* 개선 사항 :
    * 기존에 수집된 기사 주소를 저장하여 중복 요청을 방지하였다.
    * 스레드풀을 사용하여 기사 크롤링을 병렬 처리하였다.
    * 딜레이를 삽입하여 속도를 조절하였다.
* 한계 : 사이트맵 페이지는 여전히 직렬적으로 처리되었으며, 세션 관리가 부족해 동시성 문제가 발생할 수 있었다.
    

3. crawl_2.py (2차 개선)
* 개선 사항 :
    * 사이트맵 페이지와 기사 본문을 모두 병렬 처리할 수 있도록 구조를 바꾸었다.
    * 각 스레드가 독립적인 세션을 사용하도록 하여 경합 문제를 예방하였다.
    * 파일 기록 과정에서 동기화를 적용하여 JSON 저장 충돌을 방지하였다.
    

4. crawl_3.py (3차 개선)
* 개선 사항 :
    * 기존에 수집된 파일을 불러와 중복된 기사 주소를 건너뛰도록 하였다.
    * 기사 병렬 처리 수를 크게 늘려 고속 수집이 가능하도록 하였다.
    * 사이트맵 페이지는 순차적으로 탐색하여 네트워크 부하를 줄였다.
* 결과 : 성능, 안정성, 중복 제거가 모두 강화되었다.
    

5. crawl_4.py (정확도 개선)
* 개선 사항 :
    * 기사 주소 패턴 기반의 날짜 추정 방식 대신, HTML 내 발행일 속성을 직접 읽어 정확도를 높였다.
    * 기존 방식과 병행하여 이중 안전장치를 두어 안정성을 확보하였다.
* 결과 : 시계열 분석에 필요한 발행 시각 정보를 정확히 확보할 수 있었다.

---

### 배운 점
1. 연합뉴스는 자동화 크롤러 차단이 강력하기 때문에 단순한 요청이나 Selenium만으로는 수집이 어렵다.
2. 검색 페이지 구조가 자주 바뀌며 동적 로딩이 많아 정적 파싱 방식은 불안정하다.
3. 대량 크롤링을 안정적으로 수행하기 위해서는 IP 차단 방지 대책과 주기적 저장 및 중복 방지 전략이 필요하다.

### 최종 결론

여러 시도 끝에 Playwright와 Stealth 모드를 이용한 방식만이 안정적으로 동작하는 방법임을 확인하였다.   
여러 차례 개선을 통해 속도, 안정성, 정확도를 확보할 수 있었으며, 앞으로는 IP 분산과 정기적 수집 전략을 적용하여 장기적인 크롤링 인프라로 발전시킬 수 있을 것이다.